In [1]:
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm
import h5py
import os

from datasets import ASCADv1
from leakage_detectors.non_learning import get_trace_means, get_sum_of_differences, get_signal_to_noise_ratio

/home/min/a/jgammell/miniconda3/envs/sca-defense/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = ASCADv1(store_in_ram=True, use_full_traces=False)
print(dataset)

ASCADv1(
	resource_path=../resources/ascad_v1/ASCAD_data/ASCAD_databases/ASCAD.h5,
	store_in_ram=True,
	train=True,
	desync=0,
	data_transform=None,
	target_transform=None
)


In [3]:
resource_path = os.path.join('..', 'resources', 'ascad_v1', 'ASCAD_data', 'ASCAD_databases', 'ATMega8515_raw_traces.h5')
with h5py.File(resource_path) as database_file:
    print(database_file.keys())
    print(database_file['traces'].shape)
    print(database_file['metadata'].shape)
    print(database_file['metadata'][0].dtype)

<KeysViewHDF5 ['metadata', 'traces']>
(60000, 100000)
(60000,)
[('plaintext', 'u1', (16,)), ('ciphertext', 'u1', (16,)), ('key', 'u1', (16,)), ('masks', 'u1', (16,))]


In [4]:
fig, axes = plt.subplots(3, 5, figsize=(20, 12))
for idx, ax in enumerate(axes.flatten()):
    trace, _ = dataset[idx]
    trace = trace.squeeze()
    ax.plot(trace)

In [5]:
mean, var = np.zeros_like(trace), np.zeros_like(trace)
for idx, (trace, _) in enumerate(tqdm(dataset)):
    mean = (1/(idx+1))*trace.squeeze() + (idx/(idx+1))*mean
for idx, (trace, _) in enumerate(tqdm(dataset)):
    var = (1/(idx+1))*(trace.squeeze() - mean)**2 + (idx/(idx+1))*var
stdev = np.sqrt(var)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 127344.24it/s]


In [6]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].plot(mean)
axes[1].plot(stdev)
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Mean')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Std. deviation')

Text(0, 0.5, 'Std. deviation')

In [7]:
trace_means = get_trace_means(dataset)
sod_mask = get_sum_of_differences(dataset, trace_means=trace_means)
snr_mask = get_signal_to_noise_ratio(dataset, trace_means=trace_means)

In [8]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].plot(sod_mask.squeeze())
axes[1].plot(snr_mask.squeeze())

In [11]:
target_variables = ['subbytes', 'masked_subbytes', 'r_out', 'r_in', 'r']
target_bytes = [0, 2]
fig, axes = plt.subplots(len(target_bytes), len(target_variables), figsize=(4*len(target_variables), 4*len(target_bytes)))
progress_bar = tqdm(total=len(target_bytes)*len(target_variables))

for tb_idx, target_byte in enumerate(target_bytes):
    axes_r = axes[tb_idx, :]
    for tv_idx, target_variable in enumerate(target_variables):
        dataset.select_target(variables=target_variable, bytes=target_byte)
        ax = axes_r[tv_idx]
        snr_mask = get_signal_to_noise_ratio(dataset)
        ax.plot(snr_mask)
        ax.set_title(f'{target_variable}(byte={target_byte})')
        progress_bar.update(1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.40it/s]